# Naver 영화댓글 분류

# Huggingface Dataset 패키지
- 허깅페이스 허브에 공유된 데이터셋을  다운로드해서 전처리 및 관리할 수있도록 돕는 라이브러리. 
- 많은 공개데이터셋을 동일한 인터페이스로 사용할 수있다.
- 설치
    - `pip install datasets`
- https://huggingface.co/datasets
- https://github.com/huggingface/datasets
      
## Huggingface Dataset loading
- datasets 로딩
    - `load_data('dataset name')`
        - huggingface datasets에 등록된 Dataset 이름 넣어 Loading한다.
          
![img](figures/huggingface_dataset.png)

In [1]:
from datasets import load_dataset

nsmc = load_dataset("e9t/nsmc", trust_remote_code=True)
print(type(nsmc))
# trust_remote_code=True: 프로그램(.py)를 받아서 실행해서 데이터를 받는 경우 True로 지정.

README.md:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

C:\Users\Playdata\miniconda3\envs\ml\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\datasets--e9t--nsmc. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


nsmc.py:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

<class 'datasets.dataset_dict.DatasetDict'>


In [2]:
nsmc

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [4]:
nsmc_train = nsmc['train']
print(type(nsmc_train))
nsmc_train

<class 'datasets.arrow_dataset.Dataset'>


Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 150000
})

In [9]:
nsmc_train['id'][:5]
nsmc_train["document"][:5]
nsmc_train['label'][:5]

[0, 1, 0, 0, 1]

In [11]:
# 전체 데이터를 분리
train_X = nsmc["train"]["document"]
train_y = nsmc['train']['label']
test_X = nsmc['test']['document']
test_y = nsmc['test']['label']

print(type(train_X), type(train_y))

<class 'list'> <class 'list'>


In [13]:
# 일부만 sampling
# select(추출할 index들)
sample_train = nsmc['train'].shuffle().select(range(10_000))  # 15만개 중 10_000개만 추출
sample_test = nsmc['test'].shuffle().select(range(5_000))
# sample_train
train_X = sample_train["document"]
train_y = sample_train['label']
test_X = sample_test['document']
test_y = sample_test['label']

## 모델, 토크나이저 loading

- 모델 별 Model 클래스를 이용하거나 Auto class를 이용해 모델, 전처리기(tokenizer, ImageProcessor 등)을 로딩한다.
    - Huggingface에 저장된 model name을 입력해서 pretrained 모델을 loading 한다.
    - fine tuning 한 경우 모델 저장 디렉토리 경로를 넣어 pretrained 모델을 loading한다.
- AutoModel은 model name을 주면 그 모델이 학습한 base 모델에 맞는 객체를 생성해서 반환한다.
    - Auto Model은 task 별로 다양한 클래스들이 있다.
        - 클래스 이름 형식: AutoModelFor{Task형식}
        - ex) `AutoModelForObjectDetection`, `AutoModelForSequenceClassification`
    - https://huggingface.co/docs/transformers/model_doc/auto
    - 전처리기(tokenzier)는 사용하려는 모델이 사용한 전처리기를 사용해야 한다.

In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "beomi/kcbert-base" # base model : feature 추출기.(backbone network)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
# num_labels=2 : 클래스 개수 지정.
## base model로 분류 모델을 지정하면 분류기(estimator)는 학습이 안된 것을 추가해 모델을 반환.
#### 그래서 분류기가 추론한 클래스 개수를 지정해 준다.

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
# %pip install torchinfo

In [23]:
from torchinfo import summary
summary(model)

Layer (type:depth-idx)                                       Param #
BertForSequenceClassification                                --
├─BertModel: 1-1                                             --
│    └─BertEmbeddings: 2-1                                   --
│    │    └─Embedding: 3-1                                   23,040,000
│    │    └─Embedding: 3-2                                   230,400
│    │    └─Embedding: 3-3                                   1,536
│    │    └─LayerNorm: 3-4                                   1,536
│    │    └─Dropout: 3-5                                     --
│    └─BertEncoder: 2-2                                      --
│    │    └─ModuleList: 3-6                                  85,054,464
│    └─BertPooler: 2-3                                       --
│    │    └─Linear: 3-7                                      590,592
│    │    └─Tanh: 3-8                                        --
├─Dropout: 1-2                                               --
├─L

In [24]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## pytorch Dataset 생성
모델 입력으로 다음 4개 항목을 dictionary로 묶어서 제공하도록 구현한다.
1. input_ids: 입력 text 토큰을 id로 변환한 값
2. token_type_ids: 문자쌍 구분시 사용. 단일 문장: 0, 문자쌍-첫문장: 0, 두 번째 문장: 1
3. attention_mask: 실제 토큰값과 패딩구분값
4. labels: 정답 class index

1 ~ 3은 위의 train_encoding, test_encoding으로 만듬. labels은 train_data/test_data의 label 키 값 사용

In [26]:
# 토큰화
train_encoding = tokenizer(
    train_X,
    return_tensors="pt",
    padding=True  # train_X중 제일 토큰수가 많은 것에 맞춘다.
)
test_encoding = tokenizer(
    test_X,
    return_tensors="pt",
    padding=True
)

In [27]:
train_encoding['input_ids'].shape

torch.Size([10000, 108])

In [ ]:
from pprint import pprint
a = {
    "attention_mask":[[1, 1, 1], [2, 2, 2], [3, 3, 3]],
    "input_ids":     [[1, 1, 1], [2, 2, 2], [3, 3, 3]],
    "token_type_ids":[[1, 1, 1], [2, 2, 2], [3, 3, 3]],
}
pprint(tuple(a.items()))
print("-------------------------")
{key:value[0] for key, value in a.items()}  # 0번 index 조회

In [39]:
import torch
from torch.utils.data import Dataset
class NSMCDataset(Dataset):

    def __init__(self, comments, labels):
        # comments: encoding된 댓글 목록
        # labels: 정답 label
        self.comments = comments
        self.labels = labels

    def __getitem__(self, index):
        data = {key:values[index] for key, values in self.comment.items()}
        # attention_mask, input_ids, token_type_ids + labels
        data['labels'] = torch.tensor(self.labels[index], dtype=torch.int64)  # 정답을 추가.
        return data

    def __len__(self):
        return len(self.labels)
        

In [40]:
train_set = NSMCDataset(train_encoding, train_y)
test_set =NSMCDataset(test_encoding, test_y)
len(train_set), len(test_set)

(10000, 5000)

# 학습
- Transformers는 model 학습을 위해 TrainingArguments, Trainer 클래스를 제공한다.
- TrainingArguments Trainer를 위한 설정을 하는 클래스
- TrainingArguments, Trainer를 이용하면 training option, logging, gradient accumulation, mixed precision등을 쉽게 설정해 학습, 평가를 모두 진행할 수 있다.

In [41]:
from transformers import TrainingArguments, Trainer

# 학습 관련된 설정
save_dir = "models/nsmc"
logging_dir = "logging/nsmc"
train_args = TrainingArguments(
    output_dir=save_dir,        # 학습도중 모델을 저장할 디렉토리.

    num_train_epochs=1,         # 학습 에폭수
    per_device_train_batch_size=128, # train 배치수.
    per_device_eval_batch_size=128,  # 검증 배치수

    logging_dir=logging_dir,     # 학습과정 로그를 저장할 디렉토리.
    logging_steps= 50,           # 로그를 몇 step당 한번씩 남길지.

    eval_strategy="epoch",     # 검증을 언제할지 "epoch", "step", "no": X  기준.
    save_strategy="epoch",     # 모델을 언제 저장할지 기준.

    save_total_limit=1,      # 최대 몇개를 저장할지. 최신 N개를 저장.
    last_best_model_at_end=True, # 가장 성능 좋았던 모델을 저장할 지 여부.:True-save_total_limit+1
    metric_for_best_model="eval_loss", # best 모델 성능 평가 지표
    greater_is_better=False,    # 성능이 클수록 좋은 지 여부.

    push_to_hub=True, #  Hugging hub 계정에 학습 모델을 올릴 지 여부.
    hub_model_id="계정/이름" # 저장할 repository 
    hub_token="access token" # 인증 키
    report_to=None

)

SyntaxError: invalid syntax (3231374433.py, line 20)

## 모델 로드

# 추론

In [ ]:
sentence = ["이걸 영화라고 만든 거냐?", "아무 기대 없이 봤는데 재미있네.", "내가 감독이어도 이것보다 재미있게 만들겠다.", "시간이 어떻게 가는 줄 모르고 봤다."]